In [ ]:
#clean pdf
# data cleaning 1

import fitz
from tqdm import tqdm
from langchain.docstore.document import Document
import pdfplumber
import pandas as pd
import re

# clean the pdf data to text, tables (images not handled yet)
def keep_visible_lines(obj):
    """
    If the object is a ``rect`` type, keep it only if the lines are visible.

    A visible line is the one having ``non_stroking_color`` as 0.
    """
    if obj['object_type'] == 'rect':
        return obj['non_stroking_color'][0] < 0.1
    return True

def read_pdf(path):
    pdf_file = fitz.open(path)
    docs = []
    log_tables_pandas = []
    log_tables_raw = []
    problems = []
    for page_index in tqdm(range(len(pdf_file))):
        # get the page itself
        page = pdf_file[page_index]
        tables_raw = pdfplumber.open(filepath).pages[page_index].filter(keep_visible_lines).extract_tables()
        tables_pandas = [pd.DataFrame(i) for i in tables_raw]
        tables_pandas_colnames = []
        for tp in tables_pandas:
            tp.columns = tp.iloc[0,:]
            tp = tp.iloc[1:,:]
            tables_pandas_colnames.append(tp)

        try:
            tables_json = [i.to_json() for i in tables_pandas_colnames]
        except:
            problems.append(tables_pandas_colnames)
            print(page_index+1)
            tables_json = [i.to_html() for i in tables_pandas_colnames]
        docs.append(Document(page_content=page.get_text(), metadata={"filename":page_index}))
        log_tables_pandas.extend(tables_pandas)
        log_tables_raw.extend(tables_raw)
        for table in tables_json:
            docs.append(Document(page_content=table, metadata={"filename":page_index}))
        #print(page.get_text())
    return docs, log_tables_pandas, problems, log_tables_raw


def regex_chunking(docs):
    chunked = []
    for i in docs:
        chunked.extend([Document(page_content="For Property: Ava\n"+e) for e in re.split("[0-9]+\.", i.page_content)])
    return chunked


#test
filepath = "../data/faq/FAQ-Residensi AVA-2021.pdf"
docs_ava, log_tables, problems, log_tables_raw = read_pdf(filepath)
#docs_ava = regex_chunking(docs_ava)

#helper code to optimize table extraction by visual inspection
# p0 = pdfplumber.open(filepath).pages[0]
# p0_clean = p0.filter(keep_visible_lines)
# im = p0_clean.to_image()
# im.reset().debug_tablefinder()